In [1]:
import os



In [2]:
os.chdir('../')

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class  DataIngestionConfig:
    root_dir:Path
    source_URL:str
    local_data_file: Path
    unzip_dir:Path


In [4]:
from Image2Recipie.constants import *
from Image2Recipie.utils.common import read_yaml,create_directories

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\bisht\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH

    ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionConfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [6]:
import os
import urllib.request as request 
import gdown
from Image2Recipie import logger
from Image2Recipie.utils.common import get_size,format_recipe_ingredients,detect_english_text
import zipfile
import pandas as pd
import re
import numpy as np

In [ ]:
class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config = config

    def download_file(self)->str:
        try:
            dataset_url=self.config.source_URL
            zip_download_dir=self.config.local_data_file
            os.makedirs('artifacts/data_ingestion',exist_ok=True)
            logger.info(f'Downloading data from {dataset_url} into file {zip_download_dir}')
            file_id=dataset_url.split('/')[-2]
            prefix='http://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)
            logger.info(f'Download data from {dataset_url} into file {zip_download_dir}')
        except Exception as e:
            raise e
    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)
    def pre_process(self):
        df=pd.read_csv(os.path.join(self.config.unzip_dir,'cuisine_updated.csv'))
        
        food_name=[]
        for i in range(0,df['name'].shape[0]):
            if 'Recipe' in df['name'][i]:
                food_name.append(df['name'][i].split('Recipe')[0])
            else:
                food_name.append(df['name'][i].split('(')[0])

        df['name']=food_name
        df['cleaned_ingredient']=df['ingredients'].apply(format_recipe_ingredients)
        df['cleaned_ingredient']=df['cleaned_ingredient'].apply(lambda x:','.join([x.strip() for x in x.split(',')]))
        df['cleaned_ingredient']=df['cleaned_ingredient'].str.lower()
        paths=os.listdir(os.path.join(self.config.unzip_dir,'data\data'))
        image_path=sorted(paths,key=lambda x:int(x.split('.')[0]))
        df['image_path']=pd.DataFrame(np.array(image_path),columns=['column'])
        
        
        
        empty_index=df[df['ingredients']=='\n'].index
        
        df.drop(index=empty_index,inplace=True)
        df.reset_index(inplace=True)
        df.drop(columns=['index'],inplace=True)
        df.to_csv(os.path.join(self.config.unzip_dir,'cuisine_updated.csv'))
        logger.info(f'Preprocessed csv data is stored in {self.config.unzip_dir}')



        


In [ ]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    data_ingestion.pre_process()
except Exception as e:
    raise e

[2024-11-03 09:57:16,475:INFO:common:yaml file: config\config.yaml loaded successfully]
[2024-11-03 09:57:16,476:INFO:common:yaml file: params.yaml loaded successfully]
[2024-11-03 09:57:16,476:INFO:common:created directory at: artifacts]
[2024-11-03 09:57:16,476:INFO:common:created directory at: artifacts/data_ingestion]
[2024-11-03 09:57:16,912:INFO:2062471813:Preprocessed csv data is stored in artifacts/data_ingestion]
